# Evaluations
Runs through the different stages of the RAG agent and evaluates the performance of the different stages

In [3]:
import os
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from src.agents.agent_rag import RAGAgent

## Load the data and prep the dataset

In [10]:
import json

data_dir = os.path.join('..', 'ConvFinQA/data/')

train_data = json.load(open(os.path.join(data_dir, 'train.json')))

Not all the data has qa, so we need to filter out the data that doesn't have qa

In [33]:
qa_data = [
    {
        'id': data['id'],
        'question': data[qa_key]['question'],
        'answer': data[qa_key]['answer']
    }
    for data in train_data
    for qa_key in [k for k in data.keys() if k == 'qa' or k.startswith('qa_')] # account for multiple qa keys
]

In [34]:
print(len(train_data))
print(len(qa_data))

3037
3965


In [39]:
qa_data[:5]

[{'id': 'Single_JKHY/2009/page_28.pdf-3',
  'question': 'what was the percentage change in the net cash from operating activities from 2008 to 2009',
  'answer': '14.1%'},
 {'id': 'Single_RSG/2008/page_114.pdf-2',
  'question': 'what was the percent of the growth in the revenues from 2007 to 2008',
  'answer': '1.3%'},
 {'id': 'Single_AAPL/2002/page_23.pdf-1',
  'question': 'what was the percentage change in net sales from 2000 to 2001?',
  'answer': '-32%'},
 {'id': 'Single_UPS/2009/page_33.pdf-2',
  'question': 'what was the difference in percentage cumulative return on investment for united parcel service inc . compared to the s&p 500 index for the five year period ended 12/31/09?',
  'answer': '-26.16%'},
 {'id': 'Double_UPS/2009/page_33.pdf',
  'question': 'what is the roi of an investment in ups in 2004 and sold in 2006?',
  'answer': '-8.9%'}]

## Retrieval
First let's take a look at the retrieval stage

In [55]:
from src.nodes import retriever_node

In [59]:
LIMIT = 100

# only take the first LIMIT questions
limit_qa_data = qa_data[:LIMIT]

for data in limit_qa_data:
    question_dict = {'question': data['question']}
    retrieved_docs = retriever_node(question_dict)['retrieved_docs']

    retrieved_docs_ids = [doc['id'] for doc in retrieved_docs]
    if data['id'] in retrieved_docs_ids:
        data['retrieved'] = 1
    else:
        data['retrieved'] = 0

In [60]:
sum([data['retrieved'] for data in limit_qa_data])

62

In [ ]:
# split this is into recall and mrr. Check through my formatting and results to be sure I've got the right answers